In [ ]:
# 1. Brainstorm how algorithm works e.g. on paper, focusing specifically of what tyoe of numerical imput we need 
#2. Implement steps in form of  a basic python function

In [4]:
# Built-in Modules
import math
import random
import sys
import itertools
import statistics

# External Packages
import numpy
import pandas
import matplotlib
import seaborn  # for visualization
import scipy
import sklearn
import pytest
import unittest
import collections

In [7]:
# This is the code calculating how much each creditor will get from the debtor's property
# based on the ratio of over total 
def allocate_debt(debt_amount, creditor_dict):
    total_credit = sum(creditor_dict.values())
    allocation = {}
    for creditor, credit_amount in creditor_dict.items():
        allocation[creditor] = debt_amount * (credit_amount / total_credit)
    return allocation

def main():
    debt_amount = float(input("Enter the amount of property: ")) # the debtor's current property, not the amount of debt
    creditor_count = int(input("Enter the number of creditors: ")) # how many creditors the debtor owed 
    creditor_dict = {}
    for i in range(creditor_count):
        creditor = input(f"Enter creditor {i+1}: ") # input creditor's name (e.g A, B, C)
        credit_amount = float(input(f"Enter the amount credited by {creditor}: ")) # how much debt the debtor owed to each creditor
        creditor_dict[creditor] = credit_amount

    allocation = allocate_debt(debt_amount, creditor_dict)
    print("\nAllocation to Creditors:")
    for creditor, amount in allocation.items():
        print(f"{creditor}: {amount}")

if __name__ == "__main__":
    main()

Enter the amount of property:  3
Enter the number of creditors:  4
Enter creditor 1:  Gabe
Enter the amount credited by Gabe:  3
Enter creditor 2:  michelle
Enter the amount credited by michelle:  3
Enter creditor 3:  gg
Enter the amount credited by gg:  4
Enter creditor 4:  d
Enter the amount credited by d:  99



Allocation to Creditors:
Gabe: 0.08256880733944955
michelle: 0.08256880733944955
gg: 0.11009174311926606
d: 2.7247706422018347


In [5]:
class Dinics:
    class Edge:
        def __init__(self, from_, to, capacity, cost=0):
            self.from_ = from_
            self.to = to
            self.capacity = capacity
            self.flow = 0
            self.cost = cost
            self.original_cost = cost

        def is_residual(self):
            return self.capacity == 0

        def remaining_capacity(self):
            return self.capacity - self.flow

        def augment(self, bottle_neck):
            self.flow += bottle_neck
            self.residual.flow -= bottle_neck

    def __init__(self, n, vertex_labels):
        self.n = n
        self.graph = [[] for _ in range(n)]
        self.vertex_labels = vertex_labels
        self.level = [0] * n
        self.solved = False
        self.max_flow = 0
        self.min_cut = [False] * n
        self.visited = [0] * n
        self.visited_token = 1
        self.edges = []

    def solve(self):
        while self.bfs():
            next_ = [0] * self.n
            flow = self.dfs(self.s, next_, math.inf)
            while flow:
                self.max_flow += flow
                flow = self.dfs(self.s, next_, math.inf)

        for i in range(self.n):
            if self.level[i] != -1:
                self.min_cut[i] = True

    def bfs(self):
        self.level = [-1] * self.n
        self.level[self.s] = 0
        queue = collections.deque([self.s])
        while queue:
            node = queue.popleft()
            for edge in self.graph[node]:
                cap = edge.remaining_capacity()
                if cap > 0 and self.level[edge.to] == -1:
                    self.level[edge.to] = self.level[node] + 1
                    queue.append(edge.to)
        return self.level[self.t] != -1

    def dfs(self, at, next_, flow):
        if at == self.t:
            return flow

        num_edges = len(self.graph[at])
        while next_[at] < num_edges:
            edge = self.graph[at][next_[at]]
            cap = edge.remaining_capacity()
            if cap > 0 and self.level[edge.to] == self.level[at] + 1:
                bottle_neck = self.dfs(edge.to, next_, min(flow, cap))
                if bottle_neck > 0:
                    edge.augment(bottle_neck)
                    return bottle_neck
            next_[at] += 1
        return 0

    def visit(self, i):
        self.visited[i] = self.visited_token

    def visited(self, i):
        return self.visited[i] == self.visited_token

    def mark_all_nodes_as_unvisited(self):
        self.visited_token += 1

    def add_edges(self, edges):
        if edges is None:
            raise ValueError("Edges cannot be None")
        for edge in edges:
            self.add_edge(edge.from_, edge.to, edge.capacity)

    def add_edge(self, from_, to, capacity):
        if capacity < 0:
            raise ValueError("Capacity cannot be less than 0")
        e1 = self.Edge(from_, to, capacity)
        e2 = self.Edge(to, from_, 0)
        e1.residual = e2
        e2.residual = e1
        self.graph[from_].append(e1)
        self.graph[to].append(e2)
        self.edges.append(e1)

    def get_graph(self):
        self.execute()
        return self.graph

    def get_edges(self):
        return self.edges

    def get_max_flow(self):
        self.execute()
        return self.max_flow

    def get_min_cut(self):
        self.execute()
        return self.min_cut

    def set_source(self, s):
        self.s = s

    def set_sink(self, t):
        self.t = t

    def recompute(self):
        self.solved = False

    def execute(self):
        if self.solved:
            return
        self.solved = True
        self.solve()

class NetworkFlowSolverBase:
    INF = 10**18 // 2

    class Edge:
        def __init__(self, from_, to, capacity, cost=0):
            self.from_ = from_
            self.to = to
            self.capacity = capacity
            self.flow = 0
            self.cost = cost
            self.original_cost = cost
            self.residual = None

        def is_residual(self):
            return self.capacity == 0

        def remaining_capacity(self):
            return self.capacity - self.flow

        def augment(self, bottle_neck):
            self.flow += bottle_neck
            self.residual.flow -= bottle_neck

    def __init__(self, n, vertex_labels):
        self.n = n
        self.graph = [[] for _ in range(n)]
        self.vertex_labels = vertex_labels
        self.min_cut = [False] * n
        self.visited = [0] * n
        self.visited_token = 1
        self.edges = []
        self.solved = False
        self.max_flow = 0
        self.min_cost = 0

    def solve(self):
        pass

    def visit(self, i):
        self.visited[i] = self.visited_token

    def visited(self, i):
        return self.visited[i] == self.visited_token

    def mark_all_nodes_as_unvisited(self):
        self.visited_token += 1

    def add_edges(self, edges):
        if edges is None:
            raise ValueError("Edges cannot be None")
        for edge in edges:
            self.add_edge(edge.from_, edge.to, edge.capacity)

    def add_edge(self, from_, to, capacity):
        if capacity < 0:
            raise ValueError("Capacity cannot be less than 0")
        e1 = self.Edge(from_, to, capacity)
        e2 = self.Edge(to, from_, 0)
        e1.residual = e2
        e2.residual = e1
        self.graph[from_].append(e1)
        self.graph[to].append(e2)
        self.edges.append(e1)

    def get_graph(self):
        self.execute()
        return self.graph

    def get_edges(self):
        return self.edges

    def get_max_flow(self):
        self.execute()
        return self.max_flow

    def get_min_cost(self):
        self.execute()
        return self.min_cost

    def get_min_cut(self):
        self.execute()
        return self.min_cut

    def set_source(self, s):
        self.s = s

    def set_sink(self, t):
        self.t = t

    def recompute(self):
        self.solved = False

    def execute(self):
        if self.solved:
            return
        self.solved = True
        self.solve()


In [6]:
# Instantiate the solver
n = 7  # Number of vertices in the graph
vertex_labels = ["Alice", "Bob", "Charlie", "David", "Ema", "Fred", "Gabe"]
solver = Dinics(n, vertex_labels)

# Add edges to the flow graph
edges = [
    (0, 1, 40),  # Edge from Alice to Bob with capacity 40
    (1, 2, 20),  # Edge from Bob to Charlie with capacity 20
    # Add more edges as needed
]
for edge in edges:
    solver.add_edge(*edge)

# Set source and sink nodes
source = 0  # Alice is the source
sink = 3    # David is the sink
solver.set_source(source)
solver.set_sink(sink)

# Solve the flow network problem
solver.solve()

# Retrieve the maximum flow
max_flow = solver.get_max_flow()
print("Maximum flow in the network:", max_flow)


Maximum flow in the network: 0
